Create a results dataframe of a bowling tournament
==

In [1]:
# import libraries
import pandas as pd

## Input after tournament

In [2]:
city = 'Eindhoven'
date = "10-08-2018" 

In [18]:
result_sheet = {}
result_sheet['Vlad'] = ['7-359/7-9/81-99-9/X7/', '9/7/639/9-9/639-9/8/9', '811/639/8/X X 72519- ', 'X 7/71X 8-8/9/X X 6- ', '817/4/549-36X X X 71 ', '366/X 71X 5163X 179/9']
result_sheet['Dymytry'] = ['9/369-X 9/X 728-9/33 ', 'X 9-8/X 9-X X 8-526/5', '9/18X 9-3-339-338-7- ', '61X X 728/X 9/6/7/9/8', '9/9/9/62X 6/16619/8/7', 'X X X 3-9/X X 413663 ']
result_sheet['Kaspareski'] = ['347-62X 8/7-537/639/9', 'X X 43427-X 9/-/X 7- ', '9/3/9/X 728-9/7272X1-', '9-637-81718-9/8/7-34 ', '717/9-9-X 128/44448- ', '8-349/7-9-9/X 5-5-X-6']
result_sheet['Don'] = ['26236/81-/X X X 9-X18', '729/7-9/X X 81X X 9/9', '9-629/2518X X 9-7181 ', 'X 817-8-81278-9/8-34 ', '9/5361X 7281X 9/9/8/8', '9/X 156-729-62-16/81 ']
result_sheet['Brunswick'] = ['7-9-6-X 8/729-X 6/71 ', '7-X 53-6317/549/8-13 ', '439-6-X 3/6331719--7 ', '2-4-7--/6---9/818-72 ', 'X 72X 71X 515-X 8/XX9', '7/6-8-9/5-X 136/8--1 ']

## Functions

In [39]:
def _valid(game):
    """Check if a game string is valid.
    A valid game string is 21 chars.

    Keyword arguments:
    game -- string of game scores
    """
    if len(game) == 21:
        return True
    return False

def count_event(game, event):
    """Count the number of an event thrown in the game
    
    Keyword arguments:
    game -- a valid game string
    event -- strike
             spare
             rotterdam
             turkey
    
    Return:
    total events -- a int of the total event count
    """
    if not _valid(game):
        raise ValueError('Oops! Not a valid game string: {}'.format(game))
        
    events = {}
    events['strike'] = 'X'
    events['spare'] = '/'
    events['rotterdam'] = '-/'
    events['turkey'] = 'XXX'
    
    if event == 'turkey':
        game = game.replace(' ','')
        
    return game.count(events[event])

def score(game):
    """Calculate the scores per frame and in total.
    
    Foreach frame there are three possible scoring rules:
    
    
    Rule A. The score for a frame is the total pins bowled over during that frame, if the number is less than ten (an open frame, or error or split depending some other rules beyond the scope of this problem).
    Rule B. If all ten pins are bowled over on the first delivery (a strike), the score for that frame is 10 + the next two deliveries.
    Rule C. If all ten pins are bowled over between the first two deliveries (a spare), the score for that frame is 10 + the next delivery.

    
    Keyword arguments:
    game -- a valid game string
    
    Return:
    results -- a dict with a list of the frames and a list of 10 elements with score tuples
    """
    if not _valid(game):
        raise ValueError('Oops! Not a valid game string: {}'.format(game))
    
    global pin_list
    
    # Break the game string in frames
    frames = _break_game_in_frames(game)
    
    # Create a pin list
    pin_list = []
    game = game.replace('-','0')
    game = game.replace(' ','')
    
    for k,v in enumerate(game):
        if v == 'X':
            v = 10
        elif v == '/':
            v = 10 - int(game[(k-1)])
        pin_list.append(int(v))
    
    score = _calculate_score()
    
    return {'frames':frames,'score':score}

# Build upon the code shown in https://gist.github.com/too/008c8dd69f68428cfdac
def _calculate_score():
    # List to return
    score_list = list()
 
    global pin_list
    score = 0
    roll_index = 0
    for frame in range(10):
        if _is_strike(roll_index):
            score += 10 + pin_list[roll_index+1] + pin_list[roll_index+2]
            score_list.append((pin_list[roll_index+1] + pin_list[roll_index+2], score))
            roll_index += 1
        elif _is_spare(roll_index):
            score += 10 + pin_list[roll_index+2]
            score_list.append((10 + pin_list[roll_index+2], score))
            roll_index += 2
        else:
            score += pin_list[roll_index] + pin_list[roll_index+1]
            score_list.append((pin_list[roll_index] + pin_list[roll_index+1], score))
            roll_index += 2
    return score_list

def _is_spare(roll_index):
    global pin_list
    return pin_list[roll_index] + pin_list[roll_index+1] == 10

def _is_strike(roll_index):
    global pin_list
    return pin_list[roll_index] == 10

def _break_game_in_frames(game):
    n = 2
    frames = [game[i:i+n] for i in range(0, len(game)-3, n)]
    frames.append(game[-3:])
    return frames

## Calculate the scores per game

In [42]:
# calculates scores from the sheet_dict_string
scores = list()
player_game_result = list()

for player in list(result_sheet.keys()):
    for key, game in enumerate(result_sheet[player]):
    
        result = score(game)

        player_game_dict = {}
        player_game_dict['player'] = player
        player_game_dict['game'] = key
        player_game_dict['hundred'] = None
        
        # This could be done in one function call that returns the values
        player_game_dict['strikes']= count_event(game, 'strike')
        player_game_dict['spares']= count_event(game, 'spare')
        player_game_dict['rotterdam']= count_event(game, 'rotterdam')
        player_game_dict['turkey'] = count_event(game, 'turkey')
        
        player_game_dict['city'] = city
        player_game_dict['date'] = date
        player_game_dict['result'] = result['score'][9][1]

        for k,v in enumerate(result['frames']):
            my_dict = {}
            my_dict['player'] = player
            my_dict['game'] = key
            my_dict['city'] = city
            my_dict['date'] = date
            my_dict['frame'] = k+1
            my_dict['thrown'] = v
            my_dict['points'] = result['score'][k][0]
            my_dict['cumulative'] = result['score'][k][1]

            if result['score'][k][1] > 99 and player_game_dict['hundred'] is None:
                player_game_dict['hundred'] = my_dict['frame'] = k+1

            scores.append(my_dict)

        player_game_result.append(player_game_dict)
        
# generate dataframes
frames_df = pd.DataFrame(data=scores, columns=scores[0].keys()) 
player_game_df = pd.DataFrame(data=player_game_result, columns=player_game_result[0].keys())

In [104]:
# Position per player
gr = player_game_df.sort_values(by=['result', 'strikes'], ascending=False).groupby('game')
player_game_df['position'] = gr.cumcount()+1

In [ ]:
# Find Beerframes

# grouped_game_0 = frames_df.query("game == 0").groupby(['frame', 'player'])['thrown'].max().reset_index()
# matrix_game_0 = grouped_game_0.pivot(index='player', columns='frame', values='thrown')
# matrix_game_0

# def check_beerframe(col):
#     total_strikes = [True for i in col.get_values() if i[0] == "X"]
#     if len(total_strikes) == len(col.get_values())-1:
#         return True
#     return False

# matrix_game_0.apply(lambda col: check_beerframe(col), axis=0)

## Write the dataframes to .csv files for presentation in Power BI

In [106]:
frames_df.to_csv('result_data/{}_{}_FRAMES.csv'.format(city, date))
player_game_df.to_csv('result_data/{}_{}_GAME.csv'.format(city, date),float_format='%.f')